In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [1]:
!git clone --branch X20_analysis https://github.com/ShulzhenkoPetr/PSC-Pattern-Classification.git

Cloning into 'PSC-Pattern-Classification'...
remote: Enumerating objects: 130, done.
remote: Counting objects: 100% (130/130), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 130 (delta 39), reused 99 (delta 20), pack-reused 0
Receiving objects: 100% (130/130), 3.26 MiB | 6.86 MiB/s, done.
Resolving deltas: 100% (39/39), done.


In [ ]:
!unzip '/content/gdrive/MyDrive/Data /marketdata.db.zip' -d '/content/PSC-Pattern-Classification/'

Archive:  /content/gdrive/MyDrive/Data /marketdata.db.zip
  inflating: /content/PSC-Pattern-Classification/marketdata.db  
  inflating: /content/PSC-Pattern-Classification/__MACOSX/._marketdata.db  


In [2]:
%cd PSC-Pattern-Classification/

/content/PSC-Pattern-Classification


In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=b3b0e05458887daa663b19437b8a73fcecb3e94a5f27aa32e0201e44dc272e56
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40725 sha256=bee562d375461119f9b9eecbf2dd9a40168bde862c635cc01745e4f418af56cb
  Stored in directory: /root/.cache/pip/wheels/d7/ca/c8/86467e7957bbbcbdf4cf4870fc7dc95e9a16404b2e3c3a98c3
Successfully built sklearn fpdf


In [ ]:
!nvidia-smi

Sun Oct 16 09:50:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   76C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
for nb in [600, 700]:                         
  print(f"--------{nb}------------") 
  for i in range(10):                  
    print(f"---------computing {i} time--------")     
    %run main.py --index_type NONFOREX --index FRA40 --nb_clusters {nb} 

In [ ]:
import pandas as pd

df1 = pd.read_csv('/content/gdrive/MyDrive/Data /FRA40_500_700.csv', header=None)
df2 = pd.read_csv('/content/PSC-Pattern-Classification/data_analysis/FRA40.csv', header=None)
df3 = pd.read_csv('/content/PSC-Pattern-Classification/data_analysis/FRA40_200.csv', header=None)
df4 = pd.read_csv('/content/PSC-Pattern-Classification/data_analysis/FRA40_500.csv', header=None)
df5 = pd.read_csv('/content/PSC-Pattern-Classification/data_analysis/FRA40_nb_cl.csv', header=None)
df6 = pd.read_csv('/content/FRA40_400_500_700-4.csv', header=None)
frames = [df1, df2, df3, df4, df5, df6]
result = pd.concat(frames)

columns = ["SYMBOL", "START_DATE", "END_DATE", "DATES", "SPREAD", "N_CLUSTERS", "PREDICTIVE_CLUSTERS", "RETURN", "MIN_PIPS", "N_TRADE", "WIN_RATE", "MAX_DRAWDOWN", "SHARPE", "PATH", "BENCHMARK_RETURN"]
result.columns = columns

result.to_csv('/content/gdrive/MyDrive/Data /FRA40_result.csv', index=False)

### Analysis of FRA40 results

In [9]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [4]:
data_fra = pd.read_csv('/content/PSC-Pattern-Classification/data_analysis/FRA40_result.csv')

In [5]:
data_fra["RETURN"].describe()

count    89.000000
mean     -6.901115
std      17.578133
min     -57.048068
25%     -17.454459
50%      -8.234304
75%       1.710323
max      41.596020
Name: RETURN, dtype: float64

In [6]:
(data_fra["RETURN"] - data_fra["BENCHMARK_RETURN"]).describe()

count    89.000000
mean    -14.530728
std      17.578133
min     -64.677682
25%     -25.084073
50%     -15.863917
75%      -5.919290
max      33.966407
dtype: float64

In [7]:
means_by_clusters = data_fra.groupby(["N_CLUSTERS"])["RETURN"].mean() 
means_by_clusters_minus_bench = means_by_clusters - data_fra["BENCHMARK_RETURN"][0]
print('Means of return - bench by:', means_by_clusters_minus_bench)

Means of return - bench by: N_CLUSTERS
100   -20.573421
200   -17.293469
300   -13.199182
400   -15.926401
500   -13.712920
600   -14.488200
700    -8.260529
Name: RETURN, dtype: float64


In [30]:
data_success = data_fra.copy()
data_success["RETURN"] = (data_fra["RETURN"] - data_fra["BENCHMARK_RETURN"]) > 0
data_by_clusters = data_success.groupby(["N_CLUSTERS"])["RETURN"].sum() / data_fra.groupby(["N_CLUSTERS"])["RETURN"].size()
data_by_clusters 

N_CLUSTERS
100    0.000000
200    0.000000
300    0.166667
400    0.100000
500    0.291667
600    0.300000
700    0.285714
Name: RETURN, dtype: float64

In [31]:
data_fra.columns

Index(['SYMBOL', 'START_DATE', 'END_DATE', 'DATES', 'SPREAD', 'N_CLUSTERS',
       'PREDICTIVE_CLUSTERS', 'RETURN', 'MIN_PIPS', 'N_TRADE', 'WIN_RATE',
       'MAX_DRAWDOWN', 'SHARPE', 'PATH', 'BENCHMARK_RETURN'],
      dtype='object')

In [33]:
data_fra.groupby(["N_CLUSTERS"]).size()

N_CLUSTERS
100    10
200    15
300     6
400    10
500    24
600    10
700    14
dtype: int64

In [32]:
data_fra.groupby(["N_CLUSTERS"])["WIN_RATE"].agg(['min', 'max'])

,min,max
N_CLUSTERS,,
100,0.400000,0.500000
200,0.450450,0.506787
300,0.444976,0.518919
400,0.427746,0.551948
500,0.248366,0.655462
600,0.338129,0.633333
700,0.449438,0.583333


In [8]:
fig = px.scatter(data_fra, x="N_CLUSTERS", y="RETURN", color="SHARPE", title="Return by N_clusters with benchmark(green line) and mean returns (black line)")

fig.add_hline(y=data_fra["BENCHMARK_RETURN"][0], line_dash="dash", line_color="green")
fig.add_hrect(y0=data_fra["RETURN"].min()-10, y1=data_fra["BENCHMARK_RETURN"][0], line_width=0, fillcolor="red", opacity=0.1)
fig.add_trace(go.Scatter(x=[100,200,300,400,500,700],
                            y=means_by_clusters_minus_bench,
                            mode="lines",
                            line=go.scatter.Line(color="black"),
                            showlegend=False))
fig.show()

### Analysis for FOREX indices

In [ ]:
indices = ['CHFJPY','EURJPY','EURTRY', 'USDCAD','EURUSD','USDCHF','AUDUSD','USDCNH','USDJPY','BTCUSD', 'GBPCAD','ETHUSD','GBPCHF','USDNOK','CADCHF','EURAUD','GBPJPY','CADJPY' ,'EURCAD','GBPNZD','USDSEK', 'EURCHF','GBPUSD', 'TRYJPY', 'USDTRY', 'EURGBP', 'USDZAR']

In [ ]:
indices_usd = [i for i in indices if 'USD' in i]
print(f"Number of idices with USD: {len(indices_usd)}")
print(indices_usd)

Number of idices with USD: 13
['USDCAD', 'EURUSD', 'USDCHF', 'AUDUSD', 'USDCNH', 'USDJPY', 'BTCUSD', 'ETHUSD', 'USDNOK', 'USDSEK', 'GBPUSD', 'USDTRY', 'USDZAR']


In [ ]:
for indx in indices_usd[:2]:
  for i in range(1,3):
    %run main.py --index_type FOREX --index {indx} --nb_clusters {i*100}